In [1]:
from NNPQC import *
from PQC import *

In [2]:
N, D_in, H, D_out = 1000, 12, 12, 12
x = torch.randn(2*N, D_in)
y = torch.randn(N, D_out)

In [3]:
model = NNPQC(D_in,H,D_out)

In [4]:
criterion = torch.nn.KLDivLoss(reduce=None)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

In [5]:
for t in range(500):
    x = torch.randn(N, D_in)
    y_pred = model(x)
    pqc = circ19()
    arr = []
    for j in range(2*N):
        v1 = pqc.get(y_pred[2*j]).getH()
        v2 = pqc.get(y_pred[2*j+1])
        fid = np.abs(v1*v2)**2;
        arr.append(fid[0,0]);
    
    haar = [];
    h = Haar_dist(a=0,b=1,name="haar");
    for i in range(reps):
        haar.append(h.ppf((i+1)/reps,pqc.num));
    n_bins = 75;
    haar_pdf = plt.hist(np.array(haar), bins=n_bins, alpha=0.5,range=(0,1))[0]/reps; 
    pqc_pdf = plt.hist(np.array(arr), bins=n_bins, alpha=0.5, range=(0,1))[0]/reps;
    loss = torch.nn.KLDivLoss(pqc_pdf,haar_pdf)
    print(t, loss.item())

    # 변화도를 0으로 만들고, 역전파 단계를 수행하고, 가중치를 갱신합니다.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

CircuitError: 'Expression cannot bind non-real or non-numeric values ({Parameter(Θ[0]): tensor(-0.2462, grad_fn=<UnbindBackward>)}).'